In [28]:
import yaml
from jinja2 import Template
from langsmith import Client


### RAG Pipeline

In [7]:
processed_context = "- a \n- b"
query = "What is a?"
prompt = f"""
    You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".
    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.

    Question: 
    {query}
    Context: 
    {processed_context}
    """

In [3]:
print(prompt)


    You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".
    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.

    Question: 
    What is a?
    Context: 
    - a 
- b
    


#### Jinja templates for prompt management & versioning


In [8]:
jinja_template = """
 You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".
    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.

    Question: 
    {{ query }}
    Context: 
    {{ processed_context }}
"""

In [9]:
template = Template(jinja_template)

In [11]:
print(template.render(query="What is a?", processed_context="- a \n- b"))


 You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".
    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.

    Question: 
    What is a?
    Context: 
    - a 
- b


In [40]:

def build_prompt_jinja(query, preprocessed_retrieved_context):
    
    prompt = f"""
    You are a helpful shopping assistant who can answer questions about the products in stock.
    You are given a question and a list of products with their descriptions.
    Your job is to answer the question based on the context.

    Instructions:
    - Answer the question based on the context only.
    - Never use the word "context" in your answer and refer to it as available products.
    - If you don't know the answer, say "I don't know".
    - If the question is not related to the context, say "I don't know".
    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.

    Question: 
    {{query}}
    Context: 
    {{preprocessed_retrieved_context}}
    """

    template = Template(prompt)
    rendered_prompt = template.render(query=query, preprocessed_retrieved_context=preprocessed_retrieved_context)
    return rendered_prompt

#### Get prompt from yaml

In [16]:
def prompt_template_config(prompt_template_path, prompt_key):
    with open(prompt_template_path, "r") as f:
        yaml_data = yaml.safe_load(f)
    prompt = yaml_data["prompts"][prompt_key]
    template = Template(prompt)
    return template



In [20]:
def build_prompt_from_yaml(query, preprocessed_retrieved_context):
    
    prompt_template_path = "./prompts/retrieval_generation.yaml"
    prompt_key = "retrieval_generation"
    template = prompt_template_config(prompt_template_path, prompt_key)

    rendered_prompt = template.render(query=query, processed_context=preprocessed_retrieved_context)
    return rendered_prompt

In [21]:
build_prompt_from_yaml("What is a?", "- a \n- b")

'You are a helpful shopping assistant who can answer questions about the products in stock.\nYou are given a question and a list of products with their descriptions.\nYour job is to answer the question based on the context.\n\nInstructions:\n- Answer the question based on the context only.\n- Never use the word "context" in your answer and refer to it as available products.\n- If you don\'t know the answer, say "I don\'t know".\n- If the question is not related to the context, say "I don\'t know".\n- The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.\n\nContext: \n- a \n- b\n\nQuestion: \nWhat is a?'

#### Prompt registery langsmith

In [48]:
prompt_key = "retreival-generation"

In [ ]:
ls_client = Client()
ls_template = ls_client.pull_prompt(prompt_key)
ls_template

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'retreival-generation', 'lc_hub_commit_hash': '9662608105b6af73a8afeb9305e7a7ef6af49cab839f86ccc895d9ffc010b291'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful shopping assistant who can answer questions about the products in stock.\nYou are given a question and a list of products with their descriptions.\nYour job is to answer the question based on the context.\n\nInstructions:\n    - Answer the question based on the context only.\n    - Never use the word "context" in your answer and refer to it as available products.\n    - If you don\'t know the answer, say "I don\'t know".\n    - If the question is not related to the context, say "I don\'t know".\n    - The answer to the question should contain detailed information about the product and returned with 

In [44]:
print(ls_template)

input_variables=['question'] input_types={} partial_variables={} metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'retreival-generation', 'lc_hub_commit_hash': '9662608105b6af73a8afeb9305e7a7ef6af49cab839f86ccc895d9ffc010b291'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful shopping assistant who can answer questions about the products in stock.\nYou are given a question and a list of products with their descriptions.\nYour job is to answer the question based on the context.\n\nInstructions:\n    - Answer the question based on the context only.\n    - Never use the word "context" in your answer and refer to it as available products.\n    - If you don\'t know the answer, say "I don\'t know".\n    - If the question is not related to the context, say "I don\'t know".\n    - The answer to the question should contain detailed information about the product and returned with detailed specifications

In [45]:
ls_template.messages[0].prompt.template


'You are a helpful shopping assistant who can answer questions about the products in stock.\nYou are given a question and a list of products with their descriptions.\nYour job is to answer the question based on the context.\n\nInstructions:\n    - Answer the question based on the context only.\n    - Never use the word "context" in your answer and refer to it as available products.\n    - If you don\'t know the answer, say "I don\'t know".\n    - If the question is not related to the context, say "I don\'t know".\n    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.\n\nContext: \n{{preprocessed_retrieved_context}}\n\nQuestion: \n{{query}}'

In [51]:
def prompt_registry(prompt_key):
    template = ls_client.pull_prompt(prompt_key)
    template = Template(template.messages[0].prompt.template)
    return template

prompt = prompt_registry(prompt_key)

prompt.render(query="What is a?", preprocessed_retrieved_context="- a \n- b")




'You are a helpful shopping assistant who can answer questions about the products in stock.\nYou are given a question and a list of products with their descriptions.\nYour job is to answer the question based on the context.\n\nInstructions:\n    - Answer the question based on the context only.\n    - Never use the word "context" in your answer and refer to it as available products.\n    - If you don\'t know the answer, say "I don\'t know".\n    - If the question is not related to the context, say "I don\'t know".\n    - The answer to the question should contain detailed information about the product and returned with detailed specifications in bullet points.\n\nContext: \n- a \n- b\n\nQuestion: \nWhat is a?'

We have 2 ways to do prompt management - yaml & langsmith